# Pst_104E_v12_post_allele_analysis_v02_RT

Based on original code by Benjamin Schwessinger.

- Inputs: output from `Pst_104E_defining_alleles_v02` & primary+haplotig (ph) protein/gene/cds .*fasta* files.
- Programs: **MUSCLE**, **PAML**
- Purpose: generate and save a DataFrame containing dN/dS information (number of nonsynonymous substitutions per non-synonymous site to the number of synonymous substitutions per synonymous site), as well as Hamming & Levenshtein distances (measures of % identity). Also provides visualisations of some of this data.

#### Overview
1. Reads in the large allele DataFrames generated in `Pst_104E_defining_alleles_v02` (i.e. proteinortho hits OR best blast hit) - see description header cell in that notebook for more information on which alleles are included in that DataFrame.
2. Filters the allele DataFrames based on %ID and %QCov (this can be set to filter only BLAST-identified alleles or both BLAST- and proteinortho-identified alleles) so that distance information is not calculated on an unnecessarily large number of alleles.
3. Calculates distance & dN/dS information, and saves this to an output file so that it does not have to be re-calculated (if for whatever reason, the inputs change so that dN/dS or distance information should change, this output file (`xx_analysed_alleles.df`) should be deleted so that it can be re-generated.
4. Plots graphs of allele-type distribution (pie chart) and allele-type Levenshtein distances (measures of similarity) for different levels of allele-filtering (QCov/TCov/%ID/Levenshtein similarity).

NB:
- dN/dS information is currently not utilised in this script.

In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import os
import shutil
from Bio import SeqIO
from Bio import AlignIO
import distance
import editdistance
import math
import subprocess
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import collections
import pybedtools
from sklearn.externals.joblib import Parallel, delayed
import itertools as it

In [47]:
GENOME_VERSION = 'Pst_104E_v12'

BASE_PATH = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/Warrior_comp_runs/allele_analysis'

YN00_PATH = os.path.join(BASE_PATH, 'post_allele_analysis', 'yn00.ctl')
BASE_OUT_PATH = os.path.join(BASE_PATH, 'post_allele_analysis')
ALLELE_PATH = os.path.join(BASE_PATH,'defining_alleles/allele_analysis/alleles_proteinortho_graph516' )
UNFILTERED_DF_PATH = os.path.join(BASE_PATH,'defining_alleles/allele_analysis',\
                '%s_p_ctg.%s_h_ctg.0.001.blastp.outfmt6.allele_analysis' % (GENOME_VERSION, GENOME_VERSION))
GENOME_PATH = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/032017_assembly'
FIGURE_PATH = os.path.join(BASE_OUT_PATH, 'figures')

GENOME = GENOME_VERSION
P_GENOME = GENOME + '_p_ctg'
H_GENOME = GENOME + '_h_ctg'
threads = 8

# Base filtering so that distance calculations are not performed on all allele pairs.
# Distance calculations will only be performed on allele pairs above the defined cutoffs.
# Note that proteinortho alleles will not be affected (this can be changed in the filterAlleleDf function).
BASE_QCOV_CUTOFF = 0
BASE_TCOV_CUTOFF = 0
BASE_PCTID_CUTOFF = 0

P_PROTEINS_FASTA = os.path.join(GENOME_PATH, P_GENOME + '.anno.protein.fa')
PH_PROTEIN_FASTA = os.path.join(GENOME_PATH, GENOME + '_ph_ctg.anno.protein.fa')
PH_GENE_FASTA = os.path.join(GENOME_PATH, GENOME + '_ph_ctg.anno.gene.fa')
PH_CDS_FASTA = os.path.join(GENOME_PATH, GENOME + '_ph_ctg.anno.CDS.fa')

PAML_PATH = os.path.join(BASE_OUT_PATH, 'paml')
if not os.path.exists(BASE_OUT_PATH):
    os.mkdir(BASE_OUT_PATH)
if not os.path.exists(FIGURE_PATH):
    os.mkdir(FIGURE_PATH)
if not os.path.exists(PAML_PATH):
    os.mkdir(PAML_PATH)
    shutil.copy2(YN00_PATH, PAML_PATH)

In [21]:
#for homozygous region analysis
COV_PATH = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/COV'
homo_bed_fh = os.path.join(COV_PATH, 'Pst_104E_v12_ph_ctg.ph_p_homo_cov.bed')
anno_gff_p_fh = os.path.join(GENOME_PATH, 'Pst_104E_v12_p_ctg.sorted.anno.gff3')

In [41]:
PH_PROTEIN_FASTA = os.path.join(GENOME_PATH, GENOME + '_ph_ctg.anno.protein.fa')
P_PROTEIN_FASTA = os.path.join(GENOME_PATH, GENOME + '_p_ctg.anno.protein.fa')
H_PROTEIN_FASTA = os.path.join(GENOME_PATH, GENOME + '_h_ctg.anno.protein.fa')
PH_GENE_FASTA = os.path.join(GENOME_PATH, GENOME + '_ph_ctg.anno.gene.fa')
PH_CDS_FASTA = os.path.join(GENOME_PATH, GENOME + '_ph_ctg.anno.CDS.fa')
PH_GFF_FH = os.path.join(GENOME_PATH, GENOME + '_ph_ctg.anno.gff3')

In [6]:
#this is only for Pst_104E_post_analysis starting with version v12 required as only v13 has th locus_tags attached
PH_GFF_DICT_PATH = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/gitrepo_sub_21092017/Assembly'
PH_GFF_DICT_FH = os.path.join(PH_GFF_DICT_PATH, 'Pst_104E_v13_ph_ctg.anno.gff3')

In [8]:
#for finalizing the protein ortho single analysis
proortho_fh = os.path.join(BASE_PATH, 'defining_alleles','proteinortho', 'ph_ctg_516.proteinortho')
proorthograph_fh = os.path.join(BASE_PATH,'defining_alleles', 'proteinortho', 'ph_ctg_516.proteinortho-graph')
poff_graph_fh = os.path.join(BASE_PATH, 'defining_alleles','proteinortho', 'ph_ctg_516.poff-graph')
assert(os.path.exists(proortho_fh))
assert(os.path.exists(proorthograph_fh))
assert(os.path.exists(poff_graph_fh))

In [9]:
def getInterHaplotypeParaloges(proorthograph_fh,poff_graph_fh):
    """This function compares the proteinorth (po) and po synteny output. 
    It returns all Target+Query pairs that are only found in po as list. 
    It also checks if there are pairs only found in poff."""
    graph_header = ['Target', 'Query', 'evalue_ab', 'bitscore_ab', 'evalue_ba', 'bitscore_ba', 'same_strand' , 'simscore']
    poff_df = pd.read_csv(poff_graph_fh, sep='\t', header=None, names=graph_header, comment='#' )
    po_df = pd.read_csv(proorthograph_fh, sep='\t', header=None, names=graph_header, comment='#' )
    poff_paring = (poff_df.Target+ '+' + poff_df.Query)
    po_paring = (po_df.Target+ '+' + po_df.Query)
    interhaplotypeparaloges_l = list(set(po_paring) - set(poff_paring))
    poff_only_pairs = list(set(poff_paring) - set(po_paring))
    if len(poff_only_pairs) > 0:
        print("Check poff and proteinortho output as conceptually all poff pairs should be in included in the proteinortho output.\nThese are the pairs:")
        print(poff_only_pairs)
    return interhaplotypeparaloges_l

In [10]:
def assign_INP(aTargetplusaQuery, proorthograph_fh = proorthograph_fh,poff_graph_fh = poff_graph_fh ):
    """
    Returns a series of of True/False values if a aTarget+aQuery pair are element identified as 
    Inter haplotype paraloge.
    Input:
        proorthograh_fh, proteinortho graph output file handle that has been run including the single flag
        queries, series of protein queries used for the blast search.
        poff_graph_fh, proteinortho graph output file handle that has been run including the synteny and
        single flag
        aTargetplusaQuery, series of protein aTargets + aQuiers used for the blast search.
    Outupt:
        Returns a True/False series with the aTargets + aQuiers as index.
    
    """

    interhaplotypeparaloges_l = getInterHaplotypeParaloges(proorthograph_fh,poff_graph_fh)
    interhaplotypeparaloges_s = pd.Series([False]*len(aTargetplusaQuery), index=aTargetplusaQuery)
    interhaplotypeparaloges_s.loc[interhaplotypeparaloges_s.index.isin(interhaplotypeparaloges_l)] = True
    #print(len(Unphasedgenes))
    return interhaplotypeparaloges_s

In [11]:
def getProOrthsingles(proortho_fh):
    """Parses out all proteinortho singles from a proteinortho output file.
    Returns a list of these singles."""
    with open(proortho_fh) as file:
        id_pattern = r'(evm\S*)'
        regex = re.compile(id_pattern)
        po_single_list = []
        for line in file:
            if '*' in line:
                po_single_list.append(regex.search(line).groups()[0])
            else:
                continue
    return po_single_list

In [12]:
def assign_PO_single(queries, proortho_fh=proortho_fh):
    """
    Returns a series of of True/False values if a query element identified as single.
    Input:
        proortho_fh, proteinortho output file handle that has been run including the single flag
        queries, series of protein queries used for the blast search.
    Outupt:
        Returns a True/False series with the queries as index.
    
    """

    PO_singles = getProOrthsingles(proortho_fh)
    PO_singles_s = pd.Series([False]*len(queries), index=queries)
    PO_singles_s.loc[PO_singles_s.index.isin(PO_singles)] = True
    #print(len(Unphasedgenes))
    return PO_singles_s

In [13]:
def allgffTogenegff(gff_fh, write_out=True):
    '''Converts at complete gff to a gene gff only and writtes it out.'''
    gene_gff = pd.read_csv(gff_fh, sep='\t', header=None)
    gene_gff = gene_gff[gene_gff[2] == 'gene']
    gene_gff.reset_index(drop=True, inplace=True)
    gene_gff.to_csv(gff_fh.replace('anno', 'gene'), sep='\t', header=False, index=False)
    return gene_gff

In [14]:
def col_8_id(x):
    '''Function that pulls out the ID from the 9th column of a df.'''
    pattern = r'ID=([a-zA-Z0-9_.]*);'
    regex = re.compile(pattern)  
    m = regex.search(x)
    match = m.groups()[0].replace('TU', 'model')
    if match.startswith('cds.'):
        match = match[4:]
    if 'exon' in match:
        _list = match.split('.')
        match = '.'.join(_list[:-1])
    return match

In [15]:
def assignMatchType(allele_source, overlap, no_overlap):
    if pd.isnull(allele_source):
        return allele_source
    
    s = allele_source + '_'
    
    if overlap ==  True:
        s += 'overlap'
    elif no_overlap == True:
        s += 'no_overlap'
    else: # different_pcontig
        s += 'unlinked'
    return s

def reduceGroups(g):
    '''returns the best hit based on e-value and BitScore per group'''
    if len(g) == 1:
        return g
    tmp_g = g[g['e-value'] == g['e-value'].min()]
    if len(tmp_g) == 1:
        return tmp_g
    return tmp_g[tmp_g['BitScore'] == tmp_g['BitScore'].max()]

def filterAlleleDf(alleleDf, qCov, tCov, pctId, levSim, leavePO=False):
    if leavePO:
        no_PO_df = alleleDf[(alleleDf['allele_source'] == 'h_rBLAST') | (alleleDf['allele_source'] == 'BLAST')]
        PO_df = alleleDf[alleleDf['allele_source'] == 'PO']

        filtered_no_PO_df = filterAlleleDf(no_PO_df, qCov, tCov, pctId, levSim)
        return filtered_no_PO_df.append(PO_df, ignore_index=True)
    
    if qCov:
        alleleDf = alleleDf[alleleDf['QCov'] > qCov]
    if tCov:
        alleleDf = alleleDf[alleleDf['TCov'] > tCov]
    if pctId:
        alleleDf = alleleDf[alleleDf['PctID'] > pctId]
    if levSim:
        levDist = (100-levSim)/100.0
        alleleDf = alleleDf[alleleDf['protein_levenshtein'] < levDist]

    return alleleDf

In [16]:
def geneUnphased(Gene_gff_fh, Homo_cov_bed_fh ):
    """
    Returns a list of all genes that are unphased.
    
    Input: * Fh for annotation gff file
           * Fh for Homo_cov_bed_fh
    Output: A set of gene IDs that are unphases
    """
    geneGff_bed = pybedtools.BedTool(Gene_gff_fh)
    homo_p_bed = pybedtools.BedTool(Homo_cov_bed_fh)
    gene_ids_ph_p_homo = []
    for x in geneGff_bed.intersect(homo_p_bed, f=0.4):
        y = col_8_id(x[8])
        gene_ids_ph_p_homo.append(y)
    gene_ids_ph_p_homo = set(gene_ids_ph_p_homo)
    return gene_ids_ph_p_homo
    

In [22]:
def assign_unphased(queries, gff_fh=anno_gff_p_fh, homo_bed_fh=homo_bed_fh):
    """
    Takes a series of gene models and checks if they overlap with the
    homo_bed dataframe. The minimum overlap is f=0.4 with the intersect function.
    
    TO DO: For now the homo_bed only includes primary gene models. Should be changed.
    Input:  
        queries, pd.Series of gene names. 
        gff_fh, file handle of gff file.
        homo_bed_dfh, file handle of bed file that represents non-phased regions.
    
    """
    if '.gene' not in gff_fh: 
        _ = allgffTogenegff(gff_fh)
    Unphasedgenes = geneUnphased(gff_fh.replace('anno', 'gene'), homo_bed_fh)
    unphased_s = pd.Series([False]*len(queries), index=queries)
    unphased_s.loc[unphased_s.index.isin(Unphasedgenes)] = True
    #print(len(Unphasedgenes))
    return unphased_s

In [23]:
def getFastaDict(fastaFile):
    d = {}
    for gene in SeqIO.parse(fastaFile, 'fasta'):
        d[gene.id] = gene
    return d

In [24]:
def writeAllelicFasta(alleleOne, alleleTwo, alleleType, outPath):
    '''writes fasta file containing fasta information for two alleles
    in the outPath'''
    assert(alleleType.upper() in ['CDS', 'GENE', 'PROTEIN'])
    
    seqRecordDict = globals()['SEQRECORD_' + alleleType.upper() + '_DICT']
    try:
        alleleSeqRecords = [seqRecordDict[alleleOne], seqRecordDict[alleleTwo]]
    except KeyError:
        print(alleleOne)
        print(alleleTwo)
        print(alleleType)
        sys.exit()
    with open(os.path.join(outPath, alleleType.lower() + '.fa'), 'w') as outFile:
        SeqIO.write(alleleSeqRecords, outFile, 'fasta')
    return True

def writeAlignmentScript(alleleOutPath, scriptLoc = os.path.join(PAML_PATH, 'paml_script.sh')):
    with open(scriptLoc, 'a') as outFile:
        print('cd %s' % alleleOutPath, file=outFile)
        print('/home/gamran/anaconda3/muscle3.8.31_i86linux64 -clwstrict -in protein.fa -out protein.aln', file=outFile)
        print('perl /home/gamran/anaconda3/pal2nal.v14/pal2nal.pl -output paml protein.aln cds.fa > cds_codon.aln', file=outFile)
        print('perl /home/gamran/anaconda3/pal2nal.v14/pal2nal.pl protein.aln cds.fa > cds_codon.clustal', file=outFile)
        print('cp %s/yn00.ctl ./' % PAML_PATH, file=outFile)
        print('/home/gamran/anaconda3/paml4.9g/bin/yn00', file=outFile)
    return True

In [25]:
def prepareAlignmentBashScript(scriptLoc = os.path.join(PAML_PATH, 'paml_script.sh')):
    with open(scriptLoc, 'w') as pamlScript:
        print('#!/bin/bash', file=pamlScript)

    for index, [Query, Target] in alleleDf.iloc[:, :2].iterrows():
        #if we don't have a blast hit skip.
        if pd.isnull(Target):
            continue
        else:
            alleleOutPath = os.path.join(PAML_PATH, '%s_%s' % (Query, Target))
            if not os.path.exists(alleleOutPath):
                os.mkdir(os.path.join(PAML_PATH, '%s_%s' % (Query, Target)))

            writeAllelicFasta(Query, Target, 'CDS', alleleOutPath)
            writeAllelicFasta(Query, Target, 'PROTEIN', alleleOutPath)

            writeAlignmentScript(alleleOutPath, os.path.join(PAML_PATH, 'paml_script.sh'))

In [26]:
def assignDistancesToAlleles(folder, alignmentFile, alleleType):
    '''Adds Hamming and Levenshtein distance columns to an allele pair
    (indexed by 'folder' name) in df'''
    #print(folder)
    if pd.isnull(folder):
        return np.nan, np.nan
    assert(alleleType.upper() in ['PROTEIN', 'CDS', 'GENE'])
    seq1, seq2 = AlignIO.read(open(alignmentFile, 'r'), format='clustal', seq_count=2)
    seq1 = str(seq1.seq).upper()
    seq2 = str(seq2.seq).upper()
    assert(len(seq1) == len(seq2))
    return editdistance.eval(seq1, seq2)/len(seq1), distance.hamming(seq1, seq2, normalized=True)

def assignDistancesToAllAlleles(df_folder_index, all_folders, tmp_path, suffix):
    """
    Reads in the index that contains the folder pairings for the alignements.
    Returns a protein_df and CDS_df that contain the hamming and levenshtein distance each.
    """
    cleaned_index = [x for x in df_folder_index if x in all_folders]
    count = 0
    total = len(df_folder_index)
    percentDone = 0
    protein_lev_dict = {}
    protein_ham_dict = {}
    CDS_lev_dict = {}
    CDS_ham_dict = {}
    
    #print("Calculating distances and adding them to the allele DataFrame...")
    
    for folder in cleaned_index:
        if pd.isnull(folder):
            proteinAlignmentFile = ''
            cdsAlignmentFile = ''
        else:
            proteinAlignmentFile = os.path.join(PAML_PATH, folder, 'protein.aln')
            cdsAlignmentFile = os.path.join(PAML_PATH, folder, 'cds_codon.clustal')
        #here the nan get overwritten. This doesn't matter though as they are all
        #nan anyway.
        protein_lev_dict[folder], protein_ham_dict[folder]  = \
        assignDistancesToAlleles(folder, proteinAlignmentFile, 'PROTEIN')
        CDS_lev_dict[folder], CDS_ham_dict[folder]  = \
        assignDistancesToAlleles(folder, cdsAlignmentFile, 'CDS')

        count += 1
        #if round(count/total * 100) > percentDone:
            #percentDone = round(count/total * 100)
            #print("%s%% complete" % percentDone)
            
    newdf_columns=['protein_hamming', 'protein_levenshtein', 'cds_hamming',
       'cds_levenshtein']
    if len(protein_ham_dict) > 0:
        df = pd.DataFrame([protein_ham_dict,protein_lev_dict,CDS_ham_dict,CDS_lev_dict]).T
        df.rename(columns=dict(zip(df.columns,newdf_columns)),inplace=True)
        out_name = os.path.join(tmp_path, '%s_%s.%s' % (df.index[0],df.index[-1],suffix))
        df.round(4).to_csv(out_name, sep='\t')

In [27]:
def parse_dNdS_to_df(line, folder):
    """
    Function that parses out dN and dS of a yn00 file and calls the 
    assign_dNdS function. Therefore returns a single element
    pd.Series with folder name as index.
    """
    dN = re.findall(r'dN = [-| ]?(.*) w', line)[0]
    dS = re.findall(r'dS = [-| ]?(.*) dN', line)[0]
    return assign_dNdS(dN, dS, folder)

def assign_dNdS(dN, dS, folder):
    '''
    Function that cacluates the dN/dS ratio an returns it as a series usind the folder as index.
    Input: dN, dS, folder(name)
    Output: single element pd.Series with folder name as index.
    '''
    if float(dS) > 0:
        series = pd.Series([float(dN)/float(dS)], index=[folder])
    else:
        series = pd.Series([np.nan], index=[folder])
    return series

def assign_dNdS_to_all_alleles(folder_index, all_folders, tmp_path, suffix):
    """Function that parses out the different dN/dS ratios from a yn.out file for a list/index
    of folders that contain these yn.out files. The output dataframe is saved to tmp folder using
    the suffix.
    Input:  folder_index, list or index where to find the yn.out for each pairing.
            all_folders, are all possible folders. This is used to filter out nan and so from
                paralizing etc.
            tmp_path, is the path were 
    Output: Saved out tmp df with the suffix as file ending.
    
    """
    cleaned_index = [x for x in folder_index if x in all_folders]
    #print(cleaned_index)
    yn00_s = pd.Series([], name='yn00_dN/dS')
    LWL85_s = pd.Series([], name='LWL85_dN/dS')
    LWL85m_s = pd.Series([], name='LWL85m_dN/dS')
    LPB93_s = pd.Series([], name='LPB93_dN/dS')
    #header = ['folder','yn00_dN/dS', 'LWL85_dN/dS','LWL85m_dN/dS','LPB93_dN/dS']
    #append these list
    for folder in cleaned_index:
        alleleYn = os.path.join(PAML_PATH, folder,'yn.out')
        with open(alleleYn, 'r') as ynOut:
            #now loop over the lines and parse out stuff
            for i, line in enumerate(ynOut):
                if line.startswith('seq. seq. ') and i > 0:
                    next(ynOut) # we want the line that is two after the line starting with 'seq. seq '
                    dataLine = next(ynOut)
                    dN = dataLine.split('+-')[0].rstrip().split(' ')[-1]
                    dS = dataLine.split('+-')[1].rstrip().split(' ')[-1]
                    yn00_s = yn00_s.append(assign_dNdS(dN, dS, folder))
                elif line.startswith('LWL85:') and 'nan' not in line:
                    LWL85_s = LWL85_s.append(parse_dNdS_to_df(line, folder))
                elif line.startswith('LWL85m:') and 'nan' not in line:
                    LWL85m_s= LWL85m_s.append(parse_dNdS_to_df(line, folder))
                elif line.startswith('LPB93:') and 'nan' not in line:
                    LPB93_s =LPB93_s.append(parse_dNdS_to_df(line, folder))
                else:
                    continue
    out_df = pd.concat([yn00_s.round(4),LWL85_s.round(4),\
                        LWL85m_s.round(4), LPB93_s.round(4)],axis =1)
    new_columns = ['yn00_dN/dS','LWL85_dN/dS','LWL85m_dN/dS','LPB93_dN/dS' ]
    if len(out_df) > 0:
        out_df.rename(columns=dict(zip(out_df.columns, new_columns)), inplace=True)
        out_name = os.path.join(tmp_path, '%s_%s.%s' % (out_df.index[0],out_df.index[-1],suffix))
        out_df.to_csv(out_name, sep='\t')
    #return out_df

In [28]:
def checkPamlFilesExist(alleleDf):
    '''loops through all folder names in alleleDf.index to check if their PAML files have
    all been generated in those folders. refDict is based on the contents of a folder
    that was known to be run successfully.'''
    refDict = {'aln': 2,
     'clustal': 1,
     'ctl': 1,
     'dN': 1,
     'dS': 1,
     'fa': 2,
     'out': 1,
     'rst': 1,
     'rst1': 1,
     'rub': 1,
     't': 1}
    for file in (x for x in alleleDf.index if not pd.isnull(x)):
        if not os.path.exists(os.path.join(PAML_PATH, file)):
            return False
        discrepancies = getDiscrepancies(os.path.join(PAML_PATH, file), refDict)
        if discrepancies != '':
            print(discrepancies)
            return False
    return True

In [29]:
def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return it.zip_longest(*args, fillvalue=fillvalue)

In [30]:
def combineTmpToDf(header, suffix, tmp_path, clean=True):
    """Combines the files of a temporary folder into a dataframe based on the tmp files
    suffix. Returns combined dataframe. And cleans up if needed."""
    tmp_assigneddfs_fh = [os.path.join(tmp_path, file) for file in os.listdir(tmp_path)\
                         if file.endswith(suffix) ]
    print(len(tmp_assigneddfs_fh))
    tmp_df = pd.DataFrame(columns=header)
    for df_fh in tmp_assigneddfs_fh:
        tmp_df = pd.concat([tmp_df, pd.read_csv(df_fh, index_col = 0, sep='\t')])
    if clean == True:
        #now clean up again
        for file in tmp_assigneddfs_fh:
            os.remove(file)
    return tmp_df


In [31]:
def getIdToLocusTagDict(gff_fh, kind):
    '''function to generate a dictionary of key=ID and value=LocusTag from a gff file for
    specific kind .e.g. mRNA, exon, gene.'''
    gff_cnames = ['seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes']
    df = pd.read_csv(gff_fh, sep='\t', header=None, names=gff_cnames)
    if kind not in ['gene',  'mRNA']:
        print('this kind of feature is not supported. Please use e.g. gene or mRNA')
        return False
    
    df = df[df['type'] == kind]
    
    locusSearch = re.compile(r'^.*locus_tag=(.*?)(;|$)')
    idSearch = re.compile(r'ID=(.*?);')
    
    d = {}
    
    for attr in df['attributes']:
        val = locusSearch.match(attr).group(1)
        key = idSearch.match(attr).group(1)
        if key in d.keys():
            print("Unexpected: id key: %s already in dictionary." % key)
        d[key] = val
    return d

In [32]:
def expected_count_singles(po_fh):
    """Count the singles in a po_fh file using * as counting thingy."""
    count = 0
    with open(po_fh) as file:
        for line in file:
            if '*' in line:
                count += 1
            else:
                continue
    return count

In [33]:
def get_input_sequence_number(fh):
    """Counts the number of input fasta sequences"""
    count = 0
    with open(fh) as file:
        for line in file:
            if line.startswith('>'):
                count += 1
            else:
                continue
    return count

In [34]:
hFullAlleleDf = pd.read_csv(os.path.join(ALLELE_PATH, '%s.full_df.alleles' % H_GENOME), header=0, sep='\t')
hFullAlleleDf['matchType'] = pd.Series([np.nan]*len(hFullAlleleDf.index), index=hFullAlleleDf.index)
hFullAlleleDf['matchType'] = hFullAlleleDf.apply(lambda row: assignMatchType(row['allele_source'], row['t_contig == h_contig_overlap'], row['q_contig == t_contig']), axis=1)
pFullAlleleDf = pd.read_csv(os.path.join(ALLELE_PATH, '%s.full_df.alleles' % P_GENOME), header=0, sep='\t')
pFullAlleleDf['matchType'] = pd.Series([np.nan]*len(pFullAlleleDf.index), index=pFullAlleleDf.index)
pFullAlleleDf['matchType'] = pFullAlleleDf.apply(lambda row: assignMatchType(row['allele_source'], row['t_contig == h_contig_overlap'], row['q_contig == t_contig']), axis=1)

In [35]:
# filter out haplotig proteins that already have alleles identified by BLAST or proteinortho.
pFullAlleleDf['aQuery'] = pFullAlleleDf['Query']
pFullAlleleDf['aTarget'] = pFullAlleleDf['Target']
hFullAlleleDf['aQuery'] = hFullAlleleDf['Target']
hFullAlleleDf['aTarget'] = hFullAlleleDf['Query']

In [36]:
#problem here is that adding a string to nan gives nan. Needs a function that converts the whole column to striing
pFullAlleleDf['comp'] = pFullAlleleDf.aQuery.astype(str) + '_' + pFullAlleleDf.aTarget.astype(str)
hFullAlleleDf['comp'] = hFullAlleleDf.aQuery.astype(str) + '_' + hFullAlleleDf.aTarget.astype(str)
#now only add the hFullAlleleDf were the pairing of p and h blast hasn't been captured yet

ahFullAlleleDf = hFullAlleleDf[~(hFullAlleleDf.comp.isin(pFullAlleleDf.comp))]


#This change would require to search the unphased on the singles on aQuery and aTarget. It save on computing time
#and potentially make downstream analysis easier

In [37]:
phFullAlleleDf = pFullAlleleDf.append(ahFullAlleleDf)

In [42]:
SEQRECORD_PROTEIN_DICT = getFastaDict(PH_PROTEIN_FASTA)
SEQRECORD_GENE_DICT = getFastaDict(PH_GENE_FASTA)
SEQRECORD_CDS_DICT = getFastaDict(PH_CDS_FASTA)
PROTEINID_LOCUSTAG_DICT = getIdToLocusTagDict(PH_GFF_DICT_FH, 'mRNA')
PROTEINID_LOCUSTAG_DICT[np.nan] = np.nan #needed to add the nan key that nan 
#blast hits in aQuery/aTarget dont throw an error

In [43]:
alleleDf = phFullAlleleDf.copy()
alleleDf['folder'] = alleleDf.Query + '_' + alleleDf.Target #this relies on the fact that adding string to nan is nan
alleleDf.set_index('folder', inplace=True)
alleleDf['aQuery_LT'] = alleleDf.aQuery.apply(lambda x: PROTEINID_LOCUSTAG_DICT[x])
alleleDf['aTarget_LT'] = alleleDf.aTarget.apply(lambda x: PROTEINID_LOCUSTAG_DICT[x])
# assert(len(alleleDf) == len(overlapDf) + len(noOverlapDf) + len(diffContigDf) + len(manualAssignDf))

In [44]:
os.chdir('/home/gamran/genome_analysis/Warrior/Richard/scripts')
%run file_counting.ipynb

In [45]:
def main(alleleDf = alleleDf):
    prepareAlignmentBashScript(os.path.join(PAML_PATH, 'paml_script.sh'))
    
    # if already run before, comment out this line
    print("Checking whether all PAML files already exist in %s..." % PAML_PATH)
    if checkPamlFilesExist(alleleDf):
        print('PAML appears to have been run to completion previously. Therefore, it will not be run this time.')
    else:
        'Not all files generated by PAML appear to exist. Running PAML now (this may take some time)...'
        !bash {os.path.join(PAML_PATH, 'paml_script.sh')}
        print('PAML finished running.')

    analysedAllelesPath = os.path.join(BASE_OUT_PATH, GENOME+'_analysed_alleles.df')    
    alleleDf.to_csv(analysedAllelesPath, sep='\t')
    #dataframe where the index is not 'NaN'
    noNANdf = alleleDf.loc[~alleleDf.index.isnull(),:].copy()
    all_folders = noNANdf.index
    #generate a tmp folder for the parallized analysis
    tmp_path = os.path.join(BASE_OUT_PATH, 'tmp')
    if not os.path.exists(tmp_path):
        os.mkdir(tmp_path)
    #assign the distances
    dist_suffix = 'distdf_tmp'
    #do parallized analysi
    Parallel(n_jobs=threads)(delayed(assignDistancesToAllAlleles)(list(folder_index_list),all_folders,tmp_path, dist_suffix)\
                       for folder_index_list in grouper(noNANdf.index, 100, np.nan))
    distdf_header = ['protein_hamming', 'protein_levenshtein', 'cds_hamming',
       'cds_levenshtein']
    distdf = combineTmpToDf(distdf_header, dist_suffix, tmp_path, clean=True)
    distdf['Index'] = distdf.index
    noNANdf['Index'] = noNANdf.index
    tmp_df = pd.merge(noNANdf, distdf,how='inner')
    tmp_df.to_csv(analysedAllelesPath, sep='\t')
    
    print("Done with caculate pairwise alignment distances.")
    #pd.util.testing.assert_frame_equal(alleleDf, pd.read_csv(analysedAllelesPath, sep='\t', index_col=0))
    print("Starting to parse dN/dS ratios from yn.out file.")
    dNdS_suffix = 'dNdSdf_tmp'
    #now assign the dNdS ratios
    Parallel(n_jobs=threads)(delayed(assign_dNdS_to_all_alleles)(list(folder_index_list),all_folders,tmp_path, dNdS_suffix)\
                       for folder_index_list in grouper(noNANdf.index, 100, np.nan))
    dNdS_header = ['yn00_dN/dS','LWL85_dN/dS','LWL85m_dN/dS','LPB93_dN/dS' ]
    dNdSdf = combineTmpToDf(dNdS_header, dNdS_suffix, tmp_path, clean=True)
    dNdSdf['Index'] = dNdSdf.index
    tmp_df = pd.merge(tmp_df, dNdSdf,how='inner')
    tmp_df.to_csv(analysedAllelesPath, sep='\t')
    print('Finish to parse dN/dS ratios.')
    print('Combining Dataframe of potential allele pairs with Dataframe of blast none hits.')
    #now pull together the dataframes of hits and no-hits blast again.
    pre_col_order = tmp_df.columns
    alleleDf = pd.concat([tmp_df, alleleDf.loc[alleleDf.index.isnull(), :]], axis=0, ignore_index=True)
    alleleDf = alleleDf.loc[:,pre_col_order]
    
    print("Assigning the unphased genes.")
    unphased_aQuery_s = assign_unphased(alleleDf.aQuery)
    assert(any(unphased_aQuery_s.index == alleleDf.aQuery))
    alleleDf['unphased_aQuery'] = unphased_aQuery_s.values
    
    
    ####IN FUTURE INCOOPORATE THE FOLLOWING FOR H homozygous regions#####
    #unphased_aTarget_s = assign_unphased(alleleDf.aTarget)
    #assert(any(unphased_aTarget_s.index == alleleDf.aTarget))
    #alleleDf['unphased_aTarget'] = unphased_aTarget_s.values
    #
    #

    
    print("Assining the proteinortho single genes.")
    PO_single_aQuery_s = assign_PO_single(alleleDf.aQuery)
    assert(any(PO_single_aQuery_s.index == alleleDf.aQuery))
    alleleDf['PO_single_aQuery'] = PO_single_aQuery_s.values
    
    PO_single_aTarget_s = assign_PO_single(alleleDf.aTarget)
    assert(any(PO_single_aTarget_s.index == alleleDf.aTarget))
    alleleDf['PO_single_aTarget'] = PO_single_aTarget_s.values
    
    
    print("Assining the proteinortho inter haplome paralogoues by comparing\
    proteinortho (po) and its synteny mode output (poff)")
    alleleDf['aTarget+aQuery'] = alleleDf.aTarget + '+' + alleleDf.aQuery
    interhaplotypeparaloges_s = assign_INP(alleleDf['aTarget+aQuery'])
    assert(any(interhaplotypeparaloges_s.index == alleleDf['aTarget+aQuery'] ))
    alleleDf['PO_interhaplotype_paralogs'] = interhaplotypeparaloges_s.values
    alleleDf.drop('aTarget+aQuery', axis=1, inplace=True)
    alleleDf.reset_index(drop=True, inplace=True)
    alleleDf.to_csv(analysedAllelesPath, sep='\t')
    return alleleDf

In [48]:
if __name__ == "__main__":
    alleleDf_out = main()

Checking whether all PAML files already exist in /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/Warrior_comp_runs/allele_analysis/post_allele_analysis/paml...
PAML appears to have been run to completion previously. Therefore, it will not be run this time.
252
Done with caculate pairwise alignment distances.
Starting to parse dN/dS ratios from yn.out file.
252
Finish to parse dN/dS ratios.
Combining Dataframe of potential allele pairs with Dataframe of blast none hits.
Assigning the unphased genes.
Assining the proteinortho single genes.
Assining the proteinortho inter haplome paralogoues by comparing    proteinortho (po) and its synteny mode output (poff)
Check poff and proteinortho output as conceptually all poff pairs should be in included in the proteinortho output.
These are the pairs:
['evm.model.hcontig_069_108.16+evm.model.pcontig_078.17', 'evm.model.hcontig_022_015.1+evm.model.pcontig_012.294']


In [49]:
#now some quick tests in here
assert(len(alleleDf) == len(alleleDf_out)) #in out out dataframe have the same length
#we get the right amount of singles
assert(\
       (alleleDf_out[(alleleDf_out.PO_single_aQuery == True)]['aQuery'].unique().shape[0]\
       + alleleDf_out[(alleleDf_out.PO_single_aTarget == True)]['aTarget'].unique().shape[0])\
       == expected_count_singles(proortho_fh))
#queries and targets are the all included
assert(alleleDf_out.aTarget.dropna().unique().shape[0] == get_input_sequence_number(H_PROTEIN_FASTA))
assert(alleleDf_out.aQuery.dropna().unique().shape[0] == get_input_sequence_number(P_PROTEIN_FASTA))
print('///////\n\n\nAppears all quick tests have completed successfully!\n\n//////')

///////


Appears all quick tests have completed successfully!

//////


Thoughts on how to parse this out or handle better:
    * combine dataframes with a comp column so that repricocal hits are not done twice. 
    * use this compare column as indexing and for looping over things as well in case of pmal and distance 
    calculations
    * calculate the unphased and singles on aTarget and aQuery indepedently
    
    This way would reduce the complexity of the analysis by making everything less redundant
    
Done.

Write some test that confirm that data got pulled in correctly. E.g. all singles, all aTargets and aQueries.

This is all done for now